In [1]:
import torch
import torch.nn as nn

In [4]:
words = open('the-verdict.txt').read()

vocab = ['.'] + sorted(set(''.join(words)))
vocab_size = len(vocab)

itos = {i:s for i,s in enumerate(vocab)}
stoi = {s:i for i,s in enumerate(vocab)}

vocab_size

def encode(text):
    return [stoi[s] for s in text]

def decode(ids):
    return ''.join(itos[i] for i in ids)

In [7]:
class NGram(nn.Module):
    def __init__(self, vocab_size, block_size, n_embd):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd

        self.emb = nn.Embedding(vocab_size, n_embd)
        self.fc = nn.Linear(block_size*n_embd, vocab_size)
        
    def forward(self, x):
        # shape of x: B, T
        # print("1.", x.shape)
        x = self.emb(x)
        # print("2.", x.shape)
        # shape of x: B, T, C
        B, T, C = x.shape
        x = x.view(B, T*C)
        # print("3.", x.shape)
        # shape of x: B, T*C
        logits = self.fc(x)
        # shape of logits: B, vocab_size
        return logits

In [6]:
model = NGram(vocab_size=63, block_size=3, n_embd=2)
checkpoint = torch.load('ngram_checkpoints.pth', weights_only=True)
model.load_state_dict(checkpoint)

RuntimeError: Error(s) in loading state_dict for NGram:
	Missing key(s) in state_dict: "emb.weight", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "epoch", "model_state_dict", "optimizer_state_dict", "loss". 

In [6]:
import time
from IPython.display import clear_output

In [10]:
input_ = 'I H'
output = input_

for i in range(100):
    logits = model(torch.tensor(encode(input_)).unsqueeze(0))
    probs = torch.softmax(logits, dim=-1).squeeze()

    # ix = logits.argmax(dim=-1).item()
    ix = torch.multinomial(probs, num_samples=1).item()
    input_ = input_[1:] + decode([ix])
    output += decode([ix])
    print(output)
    clear_output(wait=True)
    time.sleep(0.1)

I Heov.


Hueleer tive ceele thad, ant sit-teel, hacolirmew! I sustan -it ans af baile sy the batd bat 
